## Definition of the Optimization Problem

    Majority of the this notebook is to formulate the problem. The main part is described as follows

### Objectives 

    options:
    
    * minimize distance between target portfolio and optimal portfolio (with constraints) 
    
    * minimize turnover
    
    * minimize trading cost
        
### Constraints

    constraints should be prioritized to achieve as many constraints as possible
    
    Basic Constraints:
    
        1, Long MKV is within 45000000 and 55000000
        2, Net dollar exposure is within 0.3 and 0.4
        3, Net beta exposure is within 0.15 and 0.25
        4, Max single position weight is 0.05
        
    Liquidity Constraint:    
    
        5, Max liquidity is 10% of stock 20D moving average volume
    
    Sub industry beta exposure constraint
    
        6, Sub industry beta exposure is within -0.005 and 0.005

### Solve Problem

    Problem(cp.Minimize(objective),  constraints)

## Structure of the Notebook

### 1. Preprocessing data to covert signals to target portfolio


### 2. Define all variables (used in cvxpy solver) in numpy array


### 3. Solve problem portfolio by portfolio iteratively


@phrag10

conda activate /home/researcher/anaconda3/envs/mc_py3

In [1]:
import pandas as pd
import pickle
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import cvxpy as cp
import warnings
import xlrd
plt.rcParams["figure.figsize"] = [15,8]
warnings.filterwarnings("ignore")
%matplotlib inline

dir_farm = r'/media/farmshare/'
dir_farm2 = r'/media/farmshare2/'

In [2]:
def fetch_pool_dump_data (list_data):
    
    dict_data = {}
    
    dir_pool_dump_2000_2015 = dir_farm2 + '_PoolDump/RUSSELL3000_CONSOLIDATED/DATA-625/20190415/2000_2015/LATEST/'
    dir_pool_dump_2016_2018 = dir_farm2 + '_PoolDump/RUSSELL3000_CONSOLIDATED/DATA-625/20190415/2016_2018/LATEST/'
    
    for _data in list_data:
        
        print (_data)
        
        df_1 = pd.read_csv(dir_pool_dump_2000_2015 + _data + '.csv', index_col=0)
        df_2 = pd.read_csv(dir_pool_dump_2016_2018 + _data + '.csv', index_col=0)
        df_3 = (df_1.append(df_2))
        df_3 = df_3.loc[~df_3.index.duplicated(keep='first')]
        
        dict_data[_data] = df_3
    
    return dict_data

def fetch_extra_dump_data (dir_dump, list_data):
    
    dict_data = {}
        
    for _data in list_data:
        
        print (_data)
                
        dict_data[_data] = pd.read_csv(dir_dump + _data + '.csv', index_col=0)
    
    return dict_data

def check_constraints(df_daily_stock_variable):

    # 1, 2, 3
    
    df_daily_stock_variable['order_mkv'] = df_daily_stock_variable['order_qty'] * df_daily_stock_variable['close_price']
    df_port = df_daily_stock_variable.groupby(['side'])[['holding_mkv', 'holding_mkv_beta_adj','order_mkv']].sum()

    df_port.loc['Net'] = df_daily_stock_variable.sum(axis=0)
    
    df_port ['target_beta_exposure'] = df_port['holding_mkv_beta_adj'] / df_port['holding_mkv']['Long']
    df_port ['target_exposure'] = df_port['holding_mkv'] / df_port['holding_mkv']['Long']

    # 4

    df_sibind_beta_exposure = df_daily_stock_variable.groupby(['gics'])['holding_mkv_beta_adj'].sum() \
                                / df_port['holding_mkv']['Long']
    
    subind_beta_exposure_range = (df_sibind_beta_exposure.min(), df_sibind_beta_exposure.max())
    
    # 5

    max_weight_long = (df_daily_stock_variable['holding_mkv'][df_daily_stock_variable['holding_mkv']>0] \
                        / (df_daily_stock_variable['holding_mkv'][df_daily_stock_variable['holding_mkv']>0].sum())).max()
    
    max_weight_short = (df_daily_stock_variable['holding_mkv'][df_daily_stock_variable['holding_mkv']<0] \
                        / (df_daily_stock_variable['holding_mkv'][df_daily_stock_variable['holding_mkv']<0].sum())).max()
    
    max_weight = max(max_weight_long, max_weight_short)
    
    return df_port, subind_beta_exposure_range, max_weight

def get_subind_constr_abs (df_daily_stock_variable, 
                           x, ls, beta, cap_l, subind_range):
    
    list_constr = []
    
    df_gics_count = df_daily_stock_variable.reset_index().groupby(['gics']).sum()[['ind_l','ind_s']]    
    list_gics = list(set(df_gics_count[df_gics_count.sum(axis=1)>0].index))
    
    for _gics in list_gics:
        
#         print (_gics)
        
        subind_ind = ((df_daily_stock_variable['gics'] == _gics) + 0).values     
        
        ub_cap = subind_range * cap_l
        lb_cap = -1 * subind_range * cap_l
 
        constr = [cp.sum(x * (beta * subind_ind * ls)) <= ub_cap,
                  cp.sum(x * (beta * subind_ind * ls)) >= lb_cap]
        
        list_constr.extend(constr)
        
    return list_constr

def int_clip(n, smallest, largest): 
    
    return max(smallest, min(n, largest))

### Parameters

In [3]:
dir_signals = dir_farm + '_JIRA/PAP/PAP-32/3.TradingSignals/output/'
signal_file = 'SI_TRADE_SIGNAL.csv'
dir_out = dir_farm2 + '/Research/mchen/reports/20190715_opt/'
dir_pooldump = dir_farm2 + '_PoolDump/RUSSELL3000_CONSOLIDATED/DATA-625/20190415/'

mkv_range_long = (45000000, 55000000)
net_dollar_range = (0.3, 0.4)
net_beta_range = (0.15, 0.25)
max_single_position_w = 0.05
liquidity_mul = 0.1
subind_range = 0.005

### 1. Preprocessing data to covert signals to target portfolio

In [4]:
dict_data = fetch_extra_dump_data(dir_farm + '_JIRA/PAP/PAP-32/3.TradingSignals/ExtraDump/', 
                                  ['_barraBetaR3K_Fast','_closeF','_subIndustryF','_volumeF'])
dict_data['pre_20d_turnover_ma'] = (dict_data['_closeF'].mul(dict_data['_volumeF'])).rolling(20, min_periods = 1).median().shift(1)
dict_data['liquidity'] = (liquidity_mul * dict_data['pre_20d_turnover_ma'].div(dict_data['_closeF'])).fillna(0).astype(int)
df_gics = pd.read_excel(dir_farm2 + '_PoolDump/GICS CODE Label Jun2019.xlsx', header = None)
df_gics['len'] = df_gics[0].apply(lambda x: len(str(x))) 
df_gics = df_gics[df_gics['len'] == 8]
dict_gics = dict(zip(df_gics[1], df_gics[0]))
dict_data['si_release_dates'] = pd.read_csv(dir_out + 'si_release_dates.csv', index_col = 0).loc['2010-01-01':'2015-12-15']['release_date'].tolist()
dict_data['si_release_dates'][dict_data['si_release_dates'].index('2011-09-10')] = '2011-09-12'
dict_data['_pool'] = pd.read_csv(dir_signals + '_R3K_BASE_INDICATOR.csv', index_col = 0).loc['2010-01-01':'2015-12-31']
dict_data['cost'] = pd.read_csv(dir_pooldump + 'CostModel/10000.0_cost_total_enter.csv', index_col = 0).loc['2010-01-01':'2015-12-31']
dict_data['borrow_fee'] = pd.read_csv(dir_pooldump + '2000_2015/LATEST/_phraBorrowingCost.csv', index_col = 0).loc['2010-01-01':'2015-12-31']

dict_data['si_release_dates'] = pd.read_csv(dir_out + 'si_release_dates.csv', index_col = 0).loc['2016-01-01':'2018-12-26']['release_date'].tolist()
dict_data['_pool'] = pd.read_csv(dir_signals + '_R3K_BASE_INDICATOR.csv', index_col = 0).loc['2016-01-01':'2018-12-31']
dict_data['cost'] = pd.read_csv(dir_pooldump + 'CostModel/10000.0_cost_total_enter.csv', index_col = 0).loc['2016-01-01':'2018-12-27']
dict_data['borrow_fee'] = pd.read_csv(dir_pooldump + '2016_2018/LATEST/_phraBorrowingCost.csv', index_col = 0).loc['2016-01-01':'2018-12-31']

_barraBetaR3K_Fast
_closeF
_subIndustryF
_volumeF


In [5]:
df_signal = pd.read_csv(dir_signals + signal_file, index_col = 0).dropna(how='all')
df_signal_long_ind = ((df_signal.multiply(dict_data['_pool']).rank(axis = 1, ascending = False) <= 1000) + 0)
df_signal_short_ind = ((df_signal.multiply(dict_data['_pool']).rank(axis = 1, ascending = True) <= 1000) + 0)
df_signal_th_sign = df_signal_long_ind.add(-1 * df_signal_short_ind, fill_value = 0)

df_signal_th_sign_cum = df_signal_th_sign.rolling(1).sum()
df_scaling_factor_long = pd.read_csv(dir_signals + '_LONG_SCALE_FACTOR_Beta1_McapNaN.csv', index_col = 0).dropna(how='all')
df_scaling_factor_short = pd.read_csv(dir_signals + '_SHORT_SCALE_FACTOR_Beta1_McapNaN.csv', index_col = 0).dropna(how='all')

In [6]:
list_dates =  dict_data['si_release_dates']
list_dates.sort()

### 2. Define all variables (used in cvxpy solver) in numpy array
### 3. Solve problem portfolio by portfolio iteratively

In [7]:
dict_daily_order = {}
df_status = pd.DataFrame(index = list_dates, columns = ['cons1','cons2','cons3'])

for _date in list_dates:

    print (_date)

    # get stock variables

    df_daily_stock_variable = pd.DataFrame(index = df_signal_th_sign_cum.columns,
                                           columns = ['cum_signal','tar_w','cur_hold_qty','ind_l','ind_s','sf_l','sf_s',
                                                      'ind_pool','gics','beta','close_price','liquidity'])

    df_daily_stock_variable['cum_signal'] = df_signal_th_sign_cum.loc[_date].fillna(0)
    df_daily_stock_variable['ind_l'] = (df_daily_stock_variable['cum_signal'] > 0) + 0
    df_daily_stock_variable['sf_l'] = df_scaling_factor_long.loc[_date]
    df_daily_stock_variable['ind_s'] = (df_daily_stock_variable['cum_signal'] < 0) + 0
    df_daily_stock_variable['sf_s'] = df_scaling_factor_short.loc[_date]
    df_daily_stock_variable['ind_pool'] = 1 # place holder to apply filter

    df_signal_l = df_daily_stock_variable['cum_signal'][df_daily_stock_variable['ind_l'] * df_daily_stock_variable['ind_pool'] == 1] \
                    * df_daily_stock_variable['sf_l']
    df_signal_s = df_daily_stock_variable['cum_signal'][df_daily_stock_variable['ind_s'] * df_daily_stock_variable['ind_pool'] == 1] \
                    * df_daily_stock_variable['sf_s']

    df_daily_stock_variable['tar_w'] = ((df_signal_l/df_signal_l.sum()) \
                                        .add(-1*df_signal_s/df_signal_s.sum(), fill_value = 0))
    df_daily_stock_variable['tar_w'] = df_daily_stock_variable['tar_w'].fillna(0).round(6)

    if _date == list_dates[0]:

        df_daily_stock_variable['cur_hold_qty'] = 0

    else:

        pre_date = list_dates[list_dates.index(_date) - 1]
        df_daily_stock_variable['cur_hold_qty'] = (dict_daily_order[pre_date]['cur_hold_qty'] + \
                                                  dict_daily_order[pre_date]['order_qty']).fillna(0)

    df_daily_stock_variable['gics'] = dict_data['_subIndustryF'].iloc[0].fillna(0).astype(int)
    df_daily_stock_variable['beta'] = dict_data['_barraBetaR3K_Fast'].loc[_date].fillna(1)
    df_daily_stock_variable['close_price'] = dict_data['_closeF'].fillna(method='ffill').loc[_date].fillna(0)
    df_daily_stock_variable['liquidity'] = dict_data['liquidity'].loc[_date].fillna(0)
    df_daily_stock_variable['cost'] = (dict_data['cost'].loc[_date].fillna(0)/10000).round(4)
    df_daily_stock_variable['borrow_fee'] = dict_data['borrow_fee'].loc[_date].fillna(0) * 10 /(100 * 250)
    df_daily_stock_variable = df_daily_stock_variable.fillna(0)
    df_daily_stock_variable = df_daily_stock_variable[(df_daily_stock_variable['cur_hold_qty'] != 0) \
                                                      | (df_daily_stock_variable['tar_w'] != 0)]

    # get port variables

    dict_daily_port_variable = {}

    df_long_cur_holding = df_daily_stock_variable[df_daily_stock_variable['cur_hold_qty']>0]
    cur_mkv_l = (df_long_cur_holding['cur_hold_qty']*(df_long_cur_holding['close_price'])).sum()
    
    # Determine Target Long Capital Based on Constraints
    dict_daily_port_variable['target_mkv_l'] = int_clip(cur_mkv_l, mkv_range_long[0], mkv_range_long[1])

    # define variables

    x = cp.Variable(df_daily_stock_variable.shape[0])
    l = (df_daily_stock_variable['ind_l'] * df_daily_stock_variable['ind_pool']).values
    s = (df_daily_stock_variable['ind_s'] * df_daily_stock_variable['ind_pool']).values
    cap_l = dict_daily_port_variable['target_mkv_l']
    price = df_daily_stock_variable['close_price'].values
    beta = df_daily_stock_variable['beta'].values.round(2)
    cur_hold_mkv = df_daily_stock_variable['cur_hold_qty'].values * price
    liquid_mkv = (df_daily_stock_variable['liquidity'].values * price).round(2)
    target_w = df_daily_stock_variable['tar_w'].values
    cap_s = sum(x * s)
    target_mkv = target_w * (cap_l * l + cap_s * s)
    ls = l + s
    not_ls = (1 - l) * (1 - s)
    exposure_beta_range = (net_beta_range[0] * cap_l, net_beta_range[1] * cap_l)    
    exposure_dollar_range = (net_dollar_range[0] * cap_l, net_dollar_range[1] * cap_l)    
    max_dollar_position = max_single_position_w * cap_l
    cost = df_daily_stock_variable['cost'].values
    borrow_fee = df_daily_stock_variable['borrow_fee'].values
    total_cost = (cost + s * borrow_fee)
    order_mkv = cp.abs(x - cur_hold_mkv)

    # obj 

    objective = cp.norm(x - target_mkv)
    #objective = cp.norm((x - cur_hold_mkv)) # minimize turnover
    #objective = cp.sum(order_mkv * total_cost) # minimize trading cost

    # constrains

    cons_basic = [cp.abs(x) <= max_dollar_position,
                  cp.sum(x * l) == cap_l,
                  (x * l) >= 1e-9, #1e-9
                  (x * s) <= -1e-9, #1e-9
                  cp.sum(x * (beta * ls)) <= exposure_beta_range[1],
                  cp.sum(x * (beta * ls)) >= exposure_beta_range[0],
                  cp.sum(x * ls) <= exposure_dollar_range[1],
                  cp.sum(x * ls) >= exposure_dollar_range[0],
                  cp.abs(x * not_ls) <= 1e-9,
                  ]  

    cons_liquid = [cp.abs(x - cur_hold_mkv) <= liquid_mkv]

    cons_subind = get_subind_constr_abs (df_daily_stock_variable, 
                                         x, ls, beta, cap_l, subind_range)
    # opt   

    try:

        prob = cp.Problem(cp.Minimize(objective),  cons_basic + cons_liquid + cons_subind)
#         prob = cp.Problem(cp.Minimize(objective),  cons_basic)
        prob.solve(verbose = False)  # feastol=1e-10, max_iters = 1000000
        print (prob.status)
        df_status.loc[_date]['cons1'] = prob.status

        if prob.status == 'optimal': opt_value = x.value

        if prob.status == 'infeasible' or prob.status =='optimal_inaccurate':

            prob = cp.Problem(cp.Minimize(objective),  cons_basic + cons_subind)
            prob.solve(verbose = False)   # feastol=1e-10
            print (prob.status)
            df_status.loc[_date]['cons2'] = prob.status
            if prob.status == 'optimal': opt_value = x.value

        if prob.status == 'infeasible' or prob.status =='optimal_inaccurate':

            prob = cp.Problem(cp.Minimize(objective),  cons_basic)
            prob.solve(verbose = False)   # feastol=1e-10
            print (prob.status)
            df_status.loc[_date]['cons3'] = prob.status
            if prob.status == 'optimal': opt_value = x.value

        if prob.status == 'infeasible' or prob.status =='optimal_inaccurate':

            beta_l = sum(cur_hold_mkv * l * beta) / sum(cur_hold_mkv * l)
            beta_s = sum(cur_hold_mkv * s * beta) / sum(cur_hold_mkv * s)
            cap_s = abs((cap_l * beta_l - 0.2 * cap_l) / beta_s)
            target_mkv = target_w * (cap_l * l + cap_s * s)
            opt_value = target_mkv

    except:

        beta_l = sum(cur_hold_mkv * l * beta) / sum(cur_hold_mkv * l)
        beta_s = sum(cur_hold_mkv * s * beta) / sum(cur_hold_mkv * s)
        cap_s = abs((cap_l * beta_l - 0.2 * cap_l) / beta_s)
        target_mkv = target_w * (cap_l * l + cap_s * s)
        opt_value = target_mkv

    # generate orders

    df_daily_stock_variable['date'] = _date
    df_daily_stock_variable.index.names = ['Ticker']
    df_daily_stock_variable['capital_opt'] = opt_value
    df_daily_stock_variable['capital_opt'] = (df_daily_stock_variable['capital_opt'] * df_daily_stock_variable['ind_l']).clip(0, np.inf) \
                                    + (df_daily_stock_variable['capital_opt'] * df_daily_stock_variable['ind_s']).clip(-np.inf, 0)

    # generate orders

    df_daily_stock_variable['target_holding_qty'] = (df_daily_stock_variable['capital_opt'] \
                                            / df_daily_stock_variable['close_price']).fillna(0)
    df_daily_stock_variable['target_holding_qty'].loc[(~np.isfinite(df_daily_stock_variable['target_holding_qty']))] = 0
    df_daily_stock_variable['target_holding_qty'] = df_daily_stock_variable['target_holding_qty'].apply(np.int)

    df_daily_stock_variable['order_qty'] = (df_daily_stock_variable['target_holding_qty'] - df_daily_stock_variable['cur_hold_qty'])

    df_daily_stock_variable['holding_mkv'] = (df_daily_stock_variable['order_qty'] + df_daily_stock_variable['cur_hold_qty']) \
                                            * df_daily_stock_variable['close_price']
    df_daily_stock_variable['holding_mkv_beta_adj'] = df_daily_stock_variable['holding_mkv'] \
                                                    * df_daily_stock_variable['beta']

    df_daily_stock_variable['side'] = df_daily_stock_variable['capital_opt'].apply(lambda x: 'Long' if x > 0 
                                                                                    else 'Short' if x <= 0
                                                                                    else '')                

    # save orders
    df_daily_stock_variable['date'] = _date

    dict_daily_order[_date] =  df_daily_stock_variable    

    dict_daily_order['status'] = df_status

2016-01-27
optimal
2016-02-09
optimal_inaccurate
optimal
2016-02-24
optimal
2016-03-09
optimal_inaccurate
optimal
2016-03-24
optimal_inaccurate
optimal
2016-04-11
optimal_inaccurate
optimal
2016-04-26
optimal
2016-05-10
optimal
2016-05-24
optimal
2016-06-09
optimal_inaccurate
optimal
2016-06-24
optimal
2016-07-12
optimal_inaccurate
optimal
2016-07-26
optimal_inaccurate
optimal
2016-08-09
optimal_inaccurate
optimal
2016-08-24
optimal
2016-09-12
optimal
2016-09-26
optimal
2016-10-11
optimal_inaccurate
optimal
2016-10-25
optimal
2016-11-09
optimal_inaccurate
optimal
2016-11-25
optimal
2016-12-09
optimal
2016-12-27
optimal_inaccurate
optimal
2017-01-11
optimal
2017-01-26
optimal_inaccurate
optimal
2017-02-09
optimal_inaccurate
optimal
2017-02-27
optimal_inaccurate
optimal
2017-03-09
optimal
2017-03-24
optimal_inaccurate
optimal
2017-04-12
optimal
2017-04-25
optimal
2017-05-09
optimal
2017-05-24
optimal
2017-06-09
optimal
2017-06-26
optimal_inaccurate
optimal
2017-07-12
optimal_inaccurate
o

# Save optimized portfolios

In [8]:
dir_output = dir_farm2 +'/Research/PORTFOLIO_OPTIMIZATION/Tradelog_2016_2018/'
list_dates = [x for x in dict_daily_order.keys() if '-' in x]
dict_date = dict(zip(list_dates[:-1], list_dates[1:]))
df_all = pd.DataFrame()

for _date in list_dates:
    
    df_temp = dict_daily_order[_date].reset_index()
    df_all = df_all.append(df_temp)

df_all['exit_date'] = df_all['date'].map(dict_date)
df_close_stack =dict_data['_closeF'].fillna(method='ffill', limit=60).stack()
df_all = df_all.set_index(['exit_date','Ticker'])
df_all['exit_price'] = df_close_stack
df_all.reset_index().to_csv(dir_output + 'orders/SI_15days_1000L_1000S_BASE_DollarEXP_0.3_0.6_2016_2018.csv')